In [1]:
%matplotlib inline
from tqdm import tqdm
import numpy as np
import pymc3 as pm
np.set_printoptions(suppress=True)

/Users/ded/anaconda3/lib/python3.6/site-packages/theano/gpuarray/dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7201 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:86:00.0)
/Users/ded/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [26]:
# Parameters
N = 2000 # True total individuals
S = 10  # Sample periods

# Arrival: 40% initially present
birth = np.concatenate(([.4], np.ones(S-7) * (1-.7)/(S-2), [.3], np.ones(S-5) * (1-.7)/(S-2)), axis=0)

# Departure: fixed 20% rate All depart at 
dep  = np.concatenate((np.random.uniform(low=0.05, high=.15, size=(S-1,)), [1.]), axis=0)

# Capture probability (propensity)
alpha = .12
beta = 8.5
prop = pm.Beta.dist(alpha, beta).random(size = N)

In [27]:
# Capture History Data Structures
CH = [] # Full CH matrix
GCH = [] # First/Last/Count CH matrix
TH = []

# Generate and Arrival Distribution lmd ~ Cat(arr)
ARRIVALS = pm.Categorical.dist(birth).random(size = N)

for i in tqdm(range(N)):
    arrival = ARRIVALS[i]
    dep_tmp = dep.copy()
    dep_tmp[:arrival] = 0
    dep_tmp = np.append([1],(1-dep_tmp).cumprod())[:-1] * dep_tmp # Based on the arrival, compute probs of departure
    departure = pm.Categorical.dist(dep_tmp).random() # Sample departure
    dur = departure - arrival + 1
    capture_history = pm.Bernoulli.dist(prop[i]).random(size=(dur,1))
    capture_history = np.concatenate((np.zeros(arrival), 
                                      np.array(capture_history), 
                                      np.zeros(S - departure - 1)), 
                                     axis = 0)
    TH.append([int(arrival), int(departure), dur, prop[i], capture_history.sum()])
    if capture_history.any():
        first = capture_history.argmax()
        last = S - capture_history[::-1].argmax() -1
        count = capture_history.sum()
        GCH.append([first, last, count])
        CH.append(capture_history)

100%|██████████| 2000/2000 [00:48<00:00, 41.11it/s]


In [28]:
import pickle
fname = 'data/vch-%d-%d-%.2f-%.2f.pkl' % (N, S, alpha, beta)
fname
with open(fname, 'wb') as f:
  pickle.dump((GCH, sorted(TH, key=lambda tup: tup[3])), f)
fname

'data/vch-2000-10-0.12-8.50.pkl'

In [ ]:
np.unique(np.array(GCH)[:, 2], return_counts = True)

In [ ]:
np.concatenate((np.ones(S-1) * .2, [1.]), axis=0)

In [7]:
np.random.uniform(low=0, high=.2, size=(S-1,))

array([0.08759995, 0.01410217, 0.16212045, 0.07005163, 0.04871504,
       0.18720399, 0.06364118, 0.16924905, 0.16394403])

In [11]:
birth

array([0.4       , 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667])

In [13]:
dep

array([0.08637196, 0.00590847, 0.03893437, 0.04836698, 0.08371534,
       0.07895228, 0.10484167, 0.00715515, 0.06057719, 1.        ])

In [25]:
np.concatenate(([.4], np.ones(S-7) * (1-.7)/(S-2), [.3], np.ones(S-5) * (1-.7)/(S-2)), axis=0)

array([0.4   , 0.0375, 0.0375, 0.0375, 0.3   , 0.0375, 0.0375, 0.0375,
       0.0375, 0.0375])

In [22]:
S-1-6

3